In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tweet-sentiment-classification-dataset/tweet_sentiment.csv


In [2]:
df = pd.read_csv('/kaggle/input/tweet-sentiment-classification-dataset/tweet_sentiment.csv')
df

,tweet,sentiment
0,The event starts at 5 PM.,neutral
1,I hate how this turned out.,negative
2,Fantastic experience!,positive
3,Fantastic experience!,positive
4,This is the worst thing ever!,negative
...,...,...
995,It’s cloudy outside.,neutral
996,Great job by the team!,positive
997,I'm so happy about the news!,positive
998,It ruined my whole day.,negative


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet      1000 non-null   object
 1   sentiment  1000 non-null   object
dtypes: object(2)
memory usage: 15.8+ KB


In [4]:
# all unique chars that occure in the tweet
all_text = ''.join(df['tweet'].astype(str))
chars = sorted(set(all_text))
print("Unique Chars")
print(''.join(chars))
print('\nToatl unique charactes:',len(chars))

Unique Chars
 !'.5AFGIJMPRSTWabcdefghijklmnoprstuvwxy’

Toatl unique charactes: 41


In [5]:
# Create a mapping from chars to integere
map1 = { ch:i for i,ch in enumerate(chars) }
map2 = { i:ch for i,ch in enumerate(chars) }

encode = lambda s: [map1[c] for c in s]# encoder: take a string, output a list of integers.
decode = lambda l: ''.join([map2[i] for i in l])# decoder: take a list of integers, output a string

print(encode("Great job by the team!"))
print(decode(encode("Great job by the team!")))

[7, 32, 20, 16, 34, 0, 25, 30, 17, 0, 17, 39, 0, 34, 23, 20, 0, 34, 20, 16, 28, 1]
Great job by the team!


* Let's now encode the entire text dataset and store it into a torch.Tensor

In [6]:
import torch 
data = torch.tensor(encode(all_text),dtype=torch.long)
print(data.shape,data.dtype)
print(data[:1000])# 1000 characters we looked at earier will to the GPT look like this

torch.Size([24608]) torch.int64
tensor([14, 23, 20,  0, 20, 36, 20, 29, 34,  0, 33, 34, 16, 32, 34, 33,  0, 16,
        34,  0,  4,  0, 11, 10,  3,  8,  0, 23, 16, 34, 20,  0, 23, 30, 37,  0,
        34, 23, 24, 33,  0, 34, 35, 32, 29, 20, 19,  0, 30, 35, 34,  3,  6, 16,
        29, 34, 16, 33, 34, 24, 18,  0, 20, 38, 31, 20, 32, 24, 20, 29, 18, 20,
         1,  6, 16, 29, 34, 16, 33, 34, 24, 18,  0, 20, 38, 31, 20, 32, 24, 20,
        29, 18, 20,  1, 14, 23, 24, 33,  0, 24, 33,  0, 34, 23, 20,  0, 37, 30,
        32, 33, 34,  0, 34, 23, 24, 29, 22,  0, 20, 36, 20, 32,  1, 14, 23, 20,
         0, 20, 36, 20, 29, 34,  0, 33, 34, 16, 32, 34, 33,  0, 16, 34,  0,  4,
         0, 11, 10,  3, 15, 23, 16, 34,  0, 16,  0, 37, 30, 29, 19, 20, 32, 21,
        35, 27,  0, 19, 16, 39,  1,  7, 32, 20, 16, 34,  0, 25, 30, 17,  0, 17,
        39,  0, 34, 23, 20,  0, 34, 20, 16, 28,  1, 15, 23, 16, 34,  0, 16,  0,
        37, 30, 29, 19, 20, 32, 21, 35, 27,  0, 19, 16, 39,  1,  8, 34,  0, 32,
        

* split data into train and val sets

In [7]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [8]:
block_size = 8
train_data[:block_size+1]

tensor([14, 23, 20,  0, 20, 36, 20, 29, 34])

In [9]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target:{target}")

when input is tensor([14]) the target:23
when input is tensor([14, 23]) the target:20
when input is tensor([14, 23, 20]) the target:0
when input is tensor([14, 23, 20,  0]) the target:20
when input is tensor([14, 23, 20,  0, 20]) the target:36
when input is tensor([14, 23, 20,  0, 20, 36]) the target:20
when input is tensor([14, 23, 20,  0, 20, 36, 20]) the target:29
when input is tensor([14, 23, 20,  0, 20, 36, 20, 29]) the target:34


In [10]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size,(batch_size,))
    x = torch.stack([data[i:i+block_size]for i in ix])
    y = torch.stack([data[i+1:i+block_size+1]for i in ix])
    return x,y

xb,yb = get_batch('train')
print('inputs')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('-----')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b,:t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")



inputs
torch.Size([4, 8])
tensor([[ 8,  2, 28,  0, 33, 30,  0, 23],
        [33,  0, 14, 35, 20, 33, 19, 16],
        [24, 18,  0, 20, 38, 31, 20, 32],
        [34,  0, 16,  0, 37, 30, 29, 19]])
targets:
torch.Size([4, 8])
tensor([[ 2, 28,  0, 33, 30,  0, 23, 16],
        [ 0, 14, 35, 20, 33, 19, 16, 39],
        [18,  0, 20, 38, 31, 20, 32, 24],
        [ 0, 16,  0, 37, 30, 29, 19, 20]])
-----
when input is [8] the target: 2
when input is [8, 2] the target: 28
when input is [8, 2, 28] the target: 0
when input is [8, 2, 28, 0] the target: 33
when input is [8, 2, 28, 0, 33] the target: 30
when input is [8, 2, 28, 0, 33, 30] the target: 0
when input is [8, 2, 28, 0, 33, 30, 0] the target: 23
when input is [8, 2, 28, 0, 33, 30, 0, 23] the target: 16
when input is [33] the target: 0
when input is [33, 0] the target: 14
when input is [33, 0, 14] the target: 35
when input is [33, 0, 14, 35] the target: 20
when input is [33, 0, 14, 35, 20] the target: 33
when input is [33, 0, 14, 35, 20, 33] 

In [11]:
print(xb) # input to the transformer

tensor([[ 8,  2, 28,  0, 33, 30,  0, 23],
        [33,  0, 14, 35, 20, 33, 19, 16],
        [24, 18,  0, 20, 38, 31, 20, 32],
        [34,  0, 16,  0, 37, 30, 29, 19]])


# Notebook is incmpoleted need to implemet more